In [93]:
import numpy as np
import pandas as pd
#think before removing data - cant just chuck it because it doesnt help

Problem: Given features predict income

Exploratory Data Analysis:
. description states that missing values in this data set are denoted with a '?'; so parse NaN = ?
. since theres no header in the dataset itself, it shouldnt be generated from one of the rows

In [94]:
adult_dataframe = pd.read_csv("data/complete-adult.csv",na_values='?',header=None)
adult_dataframe.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [95]:
adult_dataframe.shape
#for more meaningful column names
cols = ["age","work class","final weight","education","education years","marital status","occupation","relationship in family","race","sex","capital gain","capital loss","weekly hours","native country","income"]
adult_dataframe.columns = cols

checking income distribution:

In [96]:
def check_income_distribution(df):
    above_50_counter = 0
    below_50_counter = 0

    for income in df["income"]:
        if income == "<=50K":
            below_50_counter+=1
        elif income == ">50K":
            above_50_counter+=1
    #check if something faster than for loop exists
    print(below_50_counter/df.shape[0]*100,"%",above_50_counter/df.shape[0]*100,"%")
check_income_distribution(adult_dataframe)

76.07182343065395 % 23.928176569346054 %


sources of dirty values: it contain lot's of outlier, missing value, or others intentionly wrong filling. 
dealing with missing values:

In [97]:
adult_dataframe.isna().sum()
# no_of_na/adult_dataframe.shape[0]*100


age                          0
work class                2799
final weight                 0
education                    0
education years              0
marital status               0
occupation                2809
relationship in family       0
race                         0
sex                          0
capital gain                 0
capital loss                 0
weekly hours                 0
native country             857
income                       0
dtype: int64

In [98]:
adult_dataframe.info()
adult_dataframe.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   age                     48842 non-null  int64 
 1   work class              46043 non-null  object
 2   final weight            48842 non-null  int64 
 3   education               48842 non-null  object
 4   education years         48842 non-null  int64 
 5   marital status          48842 non-null  object
 6   occupation              46033 non-null  object
 7   relationship in family  48842 non-null  object
 8   race                    48842 non-null  object
 9   sex                     48842 non-null  object
 10  capital gain            48842 non-null  int64 
 11  capital loss            48842 non-null  int64 
 12  weekly hours            48842 non-null  int64 
 13  native country          47985 non-null  object
 14  income                  48842 non-null  object
dtypes:

(48842, 15)

since its not a lot of rows with unknown values, we'll drop them for now, but will keep in mind that the work class and occupation unknowns seem to be related? also should be remove nan or replace with average/most_frequent?

In [99]:
adult_df_nadrop = adult_dataframe.dropna() #this is the easiedt way to deal with it for now - if final classification is not >85 at end we can NOT do this
adult_df_nadrop.shape

(45222, 15)

In [100]:
adult_df_nadrop.head()

,age,work class,final weight,education,education years,marital status,occupation,relationship in family,race,sex,capital gain,capital loss,weekly hours,native country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [101]:
check_income_distribution(adult_df_nadrop)

75.21560302507629 % 24.78439697492371 %


outliers:

In [102]:
adult_numerical_df = adult_df_nadrop.select_dtypes(include=['int64', 'float64'])
#using statisticalk techniques to find extremes in data
adult_numerical_df.describe()

,age,final weight,education years,capital gain,capital loss,weekly hours
count,45222.000000,4.522200e+04,45222.000000,45222.000000,45222.000000,45222.000000
mean,38.547941,1.897347e+05,10.118460,1101.430344,88.595418,40.938017
std,13.217870,1.056392e+05,2.552881,7506.430084,404.956092,12.007508
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.173882e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783160e+05,10.000000,0.000000,0.000000,40.000000
75%,47.000000,2.379260e+05,13.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


99999 capital gain, along with 99 weekly hours seems a bit off, so i want to inspect them a bit further

In [103]:
# for index,row in adult_df_nadrop.iterrows():
#     if row["capital gain"] == 99999 or row["weekly hours"] == 99:
#         print(row)

print(adult_df_nadrop[adult_df_nadrop["capital gain"] > 50000].shape[0])
#the capital gain certainly doesnt make a lot of sense as theres no data between 50-100k except the 229 values at 99k
print(adult_df_nadrop[adult_df_nadrop["weekly hours"] > 95].shape[0])
#this could be valid as there are other people in that range who also work what may seem like a ridiculous number of hours. thus it would be unwise to remove them
# adult_df_avg_calc = [if record for record in adult_df["capital gain"] ]
adult_df_avg_calc = adult_df_nadrop[ adult_df_nadrop["capital gain"] < 99000 ]
cap_gain_mean_without_extreme = adult_df_avg_calc["capital gain"].mean() 
adult_df_nadrop["capital gain"].replace(99000, cap_gain_mean_without_extreme, inplace=True) #alternative to replace
adult_df = adult_df_nadrop





229
148


removing records without pay or never wrked - description also says that theres people without pay and never worked, and sincd they have (almost) no income, although theyd be classified under 50k, thats not very meaningful

In [104]:
print(adult_df.shape[0])
adult_df = adult_df[ adult_df["work class"] != "Without-pay"  ]
print(adult_df.shape[0])
#that removed abut 20 recrds
adult_df = adult_df[ adult_df["work class"] != "Never-worked" ]
print(adult_df.shape[0])


45222
45201
45201


combine categories 

cmbine Federal-gov, Local-gov, State-gov  toi just gov?

cmbine grades to primary middklke high vocationalk and the rest?
Preschool 1st-4th 5th-6th 7th-8th 9th 10th 11th 12th HS-grad  Assoc-acdm, Assoc-voc, Some-college Prof-school Bachelors Masters Doctorate

note: just pre school is equivalkent to no formalk educatin

In [105]:
combine_gov = ["Federal-gov", "Local-gov", "State-gov"]
primary = ["1st-4th", "5th-6th"]
middle = ["7th-8th", "9th"]
high = ["10th", "11th", "12th", "HS-grad"]
mbohbo = ["Assoc-acdm", "Assoc-voc", "Some-college", "Prof-school"]

#Since inplace argument is available, you don't need to copy and assign the original data frame back to itself
for gov_job in combine_gov:
    adult_df['work class'].replace(gov_job, "government", inplace=True)
# adult_df[ adult_df["work class"] == "Federal-gov" ].shape[0]
for level in primary:
    adult_df["education"].replace(level, "primary school", inplace=True)

for level in middle:
    adult_df["education"].replace(level, "middle school", inplace=True)

for level in high:
    adult_df["education"].replace(level, "high school", inplace=True)

for level in mbohbo:
    adult_df["education"].replace(level, "Associate's Degree/Diploma", inplace=True)

# print(primary+middle+high+mbohbo)

New categries:
age: continuous.
workclass: Private, Self-emp-not-inc, Self-emp-inc, government.
fnlwgt: continuous.
education: Preschool ['1st-4th', '5th-6th', '7th-8th', '9th', '10th', '11th', '12th', 'HS-grad', 'Assoc-acdm', 'Assoc-voc', 'Some-college', 'Prof-school'] Bachelors, Masters, Doctorate.
education-num: continuous.
marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
sex: Female, Male.
capital-gain: continuous.
capital-loss: continuous.
hours-per-week: continuous.
native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.


#maybe we can change income to a boolean where its true if income is greater than 50k(an attempt to simplify data)


In [106]:
adult_df["income"].replace("<=50K", False, inplace=True)
adult_df["income"].replace(">50K", True, inplace=True)
adult_df.rename(columns={'income':'income greater than 50K'}, inplace=True)

In [107]:
adult_df.head()

,age,work class,final weight,education,education years,marital status,occupation,relationship in family,race,sex,capital gain,capital loss,weekly hours,native country,income greater than 50K
0,39,government,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,False
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,False
2,38,Private,215646,high school,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,False
3,53,Private,234721,high school,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,False
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,False


#convert categories to numbers?

#dealing with income class imbalance

In [108]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
adult_df_numerical = adult_df.select_dtypes(include=['int64', 'float64'])
adult_df[adult_df_numerical.columns] = sc.fit_transform(adult_df_numerical) 
cols = ["age","work class","final weight","education","education years","marital status","occupation","relationship in family","race","sex","capital gain","capital loss","weekly hours","native country","income greater than 50K"]
# adult_df_numerical = pd.DataFrame(adult_df_numerical)
# adult_df_numerical.head()
adult_df.head()


,age,work class,final weight,education,education years,marital status,occupation,relationship in family,race,sex,capital gain,capital loss,weekly hours,native country,income greater than 50K
0,0.034521,government,-1.062328,Bachelors,1.128469,Never-married,Adm-clerical,Not-in-family,White,Male,0.142808,-0.218778,-0.078412,United-States,False
1,0.867054,Self-emp-not-inc,-1.007474,Bachelors,1.128469,Married-civ-spouse,Exec-managerial,Husband,White,Male,-0.146748,-0.218778,-2.327654,United-States,False
2,-0.041164,Private,0.245173,high school,-0.438306,Divorced,Handlers-cleaners,Not-in-family,White,Male,-0.146748,-0.218778,-0.078412,United-States,False
3,1.094108,Private,0.425732,high school,-1.221694,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,-0.146748,-0.218778,-0.078412,United-States,False
4,-0.798011,Private,1.407214,Bachelors,1.128469,Married-civ-spouse,Prof-specialty,Wife,Black,Female,-0.146748,-0.218778,-0.078412,Cuba,False


maybe exclude final weight im not sure how much that helps

Normalize numerical data

Decision Tree (CART)
max depth
min sample split

In [127]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [134]:
def evaluate_model(X, y, model):
    # cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42) , cv=cv
    scores = cross_val_score(model, X, y, scoring='accuracy', n_jobs=-1)

    return scores

In [135]:
adult_df.columns

Index(['age', 'work class', 'final weight', 'education', 'education years',
       'marital status', 'occupation', 'relationship in family', 'race', 'sex',
       'capital gain', 'capital loss', 'weekly hours', 'native country',
       'income greater than 50K'],
      dtype='object')

In [136]:
X = adult_df[['age', 'work class', 'final weight', 'education', 'education years',
       'marital status', 'occupation', 'relationship in family', 'race', 'sex',
       'capital gain', 'capital loss', 'weekly hours', 'native country']]
y = adult_df[['income greater than 50K']].to_numpy()

print(type(X))
print(X.shape)
print(type(y))
print(y.shape)

<class 'pandas.core.frame.DataFrame'>
(45201, 14)
<class 'numpy.ndarray'>
(45201, 1)


In [142]:
#encoding
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import MinMaxScaler
# steps = [('Categorical', OneHotEncoder(handle_unknown='ignore'), cate_index), ('Numerical', MinMaxScaler(), num_index)]

# from sklearn.compose import ColumnTransformer
# transformer = ColumnTransformer(steps, verbose=True)
# X = transformer.fit_transform(X)

# print(type(X))
# print(X.shape)

In [143]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(31640, 14)
(13561, 14)
(31640, 1)
(13561, 1)


In [145]:
test_model = DecisionTreeClassifier()
test_model.fit(X_train,y_train)
score = test_model.score(X_test,y_test)
print(score)
#print(f"The CART model average accuracy is: {(np.mean(scores)*100):.2f}%, with Standard Deviation: {(np.std(scores)*100):.2f}.")

ValueError: could not convert string to float: 'government'

k means/svm